<a href="https://colab.research.google.com/github/MahmoudHussienMohamed/GitExpert/blob/main/FaceRecognitionAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Ngrok and config the authtoken.**


In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | \
sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && \
echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | \
sudo tee /etc/apt/sources.list.d/ngrok.list && \
sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,867 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
!ngrok authtoken <YourAuthToken>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# **Google Drive config**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Face Recognition stuff**

In [ ]:
!pip install face-recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=0b1e658c19d6ae7af84037986a6f790de0eef2a0473c1d028ff9f9bc8f0fe7be
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
import face_recognition
import json
import numpy as np
from PIL import Image

class student:
  def __init__(self, id, name, email, encoding):
    self.id = id
    self.name = name
    self.email = email
    self.face_encoding = encoding
  def __str__(self):
    return str(self.name) + ',' + str(self.id) + ',' + str(self.email)

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
   
        return json.JSONEncoder.default(self, obj)

def encoding_of(array):
  locations = face_recognition.face_locations(array)
  face_encoding = None
  if len(locations) == 1:
    face_encoding = face_recognition.face_encodings(array, known_face_locations = locations)[0]
    return 'found-face-of-one-person', face_encoding
  else:
    return "error-face-not-found-or-many-faces", None    

def check_one_encoding(exist_encodings, check_encoding): 
  results = face_recognition.compare_faces([check_encoding], exist_encodings)
  return json.dumps(int(results[0]), cls = NumpyEncoder) # 0 for false and 1 for true

# **API**

In [ ]:
import os
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request, jsonify
import json
from PIL import Image
import pickle
app = Flask(__name__)
all_students=[]
def_path = '/content/drive/MyDrive/FaceRecognition-API-Files/'

def append_with_pickle(file_name, appendant):
  if not os.path.exists(file_name):
    with open(file_name, 'wb') as f:
      pickle.dump(appendant, f)
  else:
    content = []
    with open(file_name, 'rb') as f:
      if os.stat(file_name).st_size > 0:
        content = pickle.load(f)
    content += appendant
    with open(file_name, 'wb') as f:
      pickle.dump(content, f)

run_with_ngrok(app)

@app.route('/', methods=['GET'])
def index():
    return jsonify({'FaceRecognition-API': "Version 1.0"})

@app.route('/register', methods=["POST"])
def register():
  if 'image' not in request.files :
      return  jsonify({'error': "error-key_notfound-sent-image_file" })
  file = request.files['image']
  payload = request.form.to_dict(flat=False)
  if 'student_id' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-id" })
  if 'student_name' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-name" })
  if 'student_email' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-email" })
  img = Image.open(file.stream)
  img = img.convert('RGB')
  img=img.resize((256,256))
  response, raw = encoding_of(np.array(img))
  if("error" in response):
    return jsonify({'error': response })
  else:
    all_students.append(student(payload['student_id'][0],
                                               payload['student_name'][0],
                                               payload['student_email'][0],
                                               raw))
    return jsonify({'sucess': response})

@app.route('/check_if_exist', methods=["GET"])
def check_if_exist():
    if 'image' not in request.files :
      return  jsonify({'error': "error-key_notfound-sent-image_file" })
    file = request.files['image']
    img = Image.open(file.stream)
    img = img.convert('RGB')
    response, raw = encoding_of(np.array(img))
    if("error" in response):
      return jsonify({'error': response })
    else:
      people=[]
      for i in all_students:
        if check_one_encoding(i.face_encoding,raw)=='1':
          people.append(str(i.id))
      if len(people) > 0: 
        return jsonify({'results': people})
      else:
        return jsonify({'results': "not-found"}) 

@app.route('/save_students', methods=['POST'])
def save():
  append_with_pickle(def_path + 'all_students.pkl',
                     all_students)
  return jsonify({'success': "done-file-saved"})

@app.route('/load_students', methods=['GET'])
def load():
    if os.path.exists(def_path + 'all_students.pkl'):
      with open(def_path + 'all_students.pkl', 'rb') as f:
        if os.stat(def_path + 'all_students.pkl').st_size > 0:
          global all_students 
          all_students = pickle.load(f)
          return jsonify({'success': "done-file-loaded"})
        else:
          return jsonify({'error': "empty-file"})
    else:
      return jsonify({'error': "save-file-before-load"})

@app.route('/new_session', methods=['POST'])
def create_session():
  payload = request.form.to_dict(flat=False)
  if 'course_id' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_id" })
  if 'course_name' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_name" })
  if 'session_no' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-session_no" })
  file_name = def_path + payload['course_name'][0] + '-' + payload['course_id'][0] + '-' + payload['session_no'][0] + '.pkl'
  if os.path.exists(file_name):
    return jsonify({'error': "course-already-created"})
  f = open(file_name, 'wb')
  f.close()
  return jsonify({'sucess': "session_created"})

@app.route('/record_attendance', methods=['POST'])
def record_attendance():
  payload = request.form.to_dict(flat=False)
  if 'image' not in request.files :
      return  jsonify({'error': "error-key_notfound-sent-image_file" })
  file = request.files['image']
  if 'course_id' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_id" })
  if 'course_name' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_name" })
  if 'session_no' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-session_no" })
  file_name = def_path + payload['course_name'][0] + '-' + payload['course_id'][0] + '-' + payload['session_no'][0] + '.pkl'
  if os.path.exists(file_name):
    file = request.files['image']
    img = Image.open(file.stream)
    img = img.convert('RGB')
    response, raw = encoding_of(np.array(img))
    for i in all_students:
      if check_one_encoding(i.face_encoding, raw) == '1':
        append_with_pickle(file_name, [i])
        return jsonify({'success': f"attendance-recorded-for-{i.id}"})
    return jsonify({'error' : "not-registered"})
  return jsonify({'error': "no-course-or-no-session-was-created-before"})

@app.route('/get_attendance', methods=['GET'])
def get_attendance():
  payload = request.form.to_dict(flat=False)
  if 'course_id' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_id" })
  if 'course_name' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_name" })
  if 'session_no' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-session_no" })
  file_name = def_path + payload['course_name'][0] + '-' + payload['course_id'][0] + '-' + payload['session_no'][0] + '.pkl'
  if os.path.exists(file_name):
    content = None
    with open(file_name, 'rb') as f:
      content = pickle.load(f)
      content = [str(i) for i in content]
      return jsonify({'success': content})
  return jsonify({'error': "no-course-or-no-session-was-created-before"})

@app.route('/delete_session', methods=['POST'])
def delete_session():
  payload = request.form.to_dict(flat=False)
  if 'course_id' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_id" })
  if 'course_name' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-course_name" })
  if 'session_no' not in payload :
    return  jsonify({'error': "error-key_notfound-sent-data-with-session_no" })
  file_name = def_path + payload['course_name'][0] + '-' + payload['course_id'][0] + '-' + payload['session_no'][0] + '.pkl'
  if os.path.exists(file_name):
    os.remove(file_name)
    return jsonify({'success' : f"file-{payload['course_name'][0] + '-' + payload['course_id'][0] + '-' + payload['session_no'][0]}-deleted"})
  return jsonify({'error' : "file-not-found"})
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1ef2-34-148-72-159.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Jul/2022 02:07:53] "POST /register HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:07:57] "POST /register HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:11] "POST /register HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:21] "GET /check_if_exist HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:37] "GET /check_if_exist HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:37] "POST /save_students HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:38] "GET /load_students HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:38] "POST /new_session HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:39] "POST /new_session HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:48] "POST /record_attendance HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:53] "POST /record_attendance HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:53] "GET /get_attendance HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:54] "GET /get_attendance HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2022 02:08:54] "POST /delete